In [2]:
import numpy as np

In [152]:
L = 14
N = 100
Rho = 3
Mu = 0.2

Beta = 1
Gamma = 10
Alfa = 2
R = 7
P = 0.5

Delta = 4
A = L- Delta
J = 2

In [179]:
#initialize n lists of bits (genomes) in an array
def init_population(n, l):
    return [np.random.randint(0,2, size = l) for i in range(n)]

#pop is array of individual lists of bits (genomes)
def phenotypes(pop):
    return np.sum(pop, axis = 1)

#a and b are two parents
def uniform_recombination(a, b):
    option = list(zip(a,b))
    return [option[i][np.random.randint(0,2)] for i in range(len(a))]

#at probability mu, flip a bit at a random position of the individual
def mutate(individual, mu, l):
    if np.random.uniform(0,1) < mu:
        
        index = np.random.randint(0, l)
        v = individual[index]
        
        if v == 1:
            individual[index] = 0
        else: 
            individual[index] = 1
            
    return individual

#combine genetic algorithm steps from paper by Bagnoli et al
def evolutionary_cycle(population, rho, mu, l):
    phenos = phenotypes(population)
    offspring = []
    for i,g in enumerate(phenos):   
        
        #recombination
        temp_offspring = [uniform_recombination(population[i],population[index]) for index, partner in enumerate(phenos) if np.abs(partner - g) <= rho]
        
        #reproduction
        offspring.append(temp_offspring[np.random.randint(len(temp_offspring))])
    #mutation
    offspring = [mutate(indiv, mu, l) for indiv in offspring]
    
    return offspring


def fitness(x, y, b, gamma, j, r, p, a):
    h0 = np.exp(-((x / gamma)**b) / b)
    h = h0 - j * p * np.sum(np.exp(- np.abs((x - y)/ r)**a) / a)
                        
    return np.exp(h)

In [228]:
#model implementation in the limit where rho -> infinity
def model_cycle(pop, delta, mu, l, b, gamma, j, r, p, alpha):
    
    np.random.shuffle(pop)
    phenos = phenotypes(pop)
    offspring = []    
        
    while len(offspring) <= len(pop):
        #select initial random parent
        randomparentindex = np.random.randint(0, len(phenos))
        randomparent = phenos[randomparentindex]
        
        #select second partent randomly from neighboorhood
        yindexes = [phenosindex for phenosindex, phe in enumerate(phenos) if np.abs(phe - randomparent) <= delta]
        yindexes.remove(randomparentindex)
        secondparentindex = yindexes[np.random.randint(0, len(yindexes))]
        
#         mut = ...
        #recombination + mutation
        single_offspring = mutate(uniform_recombination(pop[randomparentindex], pop[secondparentindex]), mu, l)
        #compute average fitness
        i = yindexes[0]
        avg_fitness = np.mean([fitness(phenos[i], np.concatenate([phenos[yindexes][:i], phenos[yindexes][i:]]), b, gamma, j, r, p, alpha) for i in yindexes])
        
        fitness_offspring = fitness(np.sum(single_offspring), phenos[yindexes],  b, gamma, j, r, p, alpha)
                             
#         #reproduction
        if np.random.uniform(0,1) < fitness_offspring/avg_fitness:
            offspring.append(single_offspring)
    return offspring                            
                            
                    

In [229]:
population = init_population(N, L)

In [230]:
evolve = evolution(population, Rho, Mu, L)

In [231]:
for gen in range(100):
    
    population = model_cycle(population, Delta, Mu, L, Beta, Gamma, J, R, P, Alfa)

[[0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
 [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0],
 [0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
 [0, 0, 1,